In [1]:
#! pip install translate
import pandas as pd, numpy as np, xlsxwriter, matplotlib.pyplot as plt, seaborn as sns, os, math
from datetime import datetime as dt, timedelta
from openpyxl import load_workbook
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import dendrogram
from scipy.cluster.hierarchy import cut_tree
from notifypy import Notify
notification = Notify()
from googletrans import Translator
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
print("\n*****Required libraries imported*****")


*****Required libraries imported*****


In [131]:
states = {'AP':'Andhra Pradesh',
'AR':'Arunachal Pradesh',
'AS':'Assam',
'BR':'Bihar',
'BH':'Bihar',
'CT':'Chhattisgarh',
'CG':'Chhattisgarh',
'DL':'Delhi', 
'GA':'Goa',
'GJ':'Gujarat',
'HR':'Haryana',
'HP':'Himachal Pradesh',
'JH':'Jharkhand',
'KA':'Karnataka',
'KL':'Kerala',
'MP':'Madhya Pradesh',
'MH':'Maharashtra',
'MN':'Manipur',
'ML':'Meghalaya',
'MZ':'Mizoram',
'NL':'Nagaland',
'OR':'Odisha',
'PB':'Punjab',
'RJ':'Rajasthan',
'SK':'Sikkim',
'TN':'Tamil Nadu',
'TG':'Telangana',
'TR':'Tripura',
'UP':'Uttar Pradesh',
'UT':'Uttarakhand',
'WB':'West Bengal'}

In [3]:
'OR - SED Fund - LI'.split(' - ')

['OR', 'SED Fund', 'LI']

In [8]:
'OR - SED Fund - LI'[-2:]

'LI'

In [132]:
# Defining a function to remove extra spaces between words.
def rem_space(x):
    try:
        n = x.strip(' ').split(' ') # Remove extra spaces from begining and end then splitting the name.
    except AttributeError:
        n = str(x).strip(' ').split(' ') # Remove extra spaces from begining and end then splitting the name.
    
    name = '' # Decalring a empty name
    
    for w in n:
        if w != '': # This will execute if non empty word found
            if name == '': # This will run if "name" variable is empty
                name = w # Storing name into "name" variable.
            else: # This will run if "name" variable is not empty
                name = name + ' ' + w # Joining the names with single spaces.
    
    return name

In [133]:
def csvORexcel():
    global path
    path = "C:\\Python\\read\\"+file_Name
    try:
        if file_Name.split('.')[-1].startswith('c'):
            df = pd.read_csv(path)
            return df
        elif file_Name.split('.')[-1].startswith('x'):
            df = pd.read_excel(path)
            return df
    except FileNotFoundError:
        print("The file name {0} has not found".format(path))

fn = file_Name

In [134]:
file_Name = 'rate_card.xlsx'

In [135]:
PID = 'PID/MAXLI/2024/DI/0422'

In [136]:
rate_card = csvORexcel()
rate_card = rate_card[rate_card['Org_PID'] == PID].sort_values('created_on')
rate_card.head()

,Org_PID,Org_Name,org_id,schemes_Guid,sch_name,sch id,ratecard_guid,open_price,Docket submitted price,scheme_document_received price,created_on
68486,PID/MAXLI/2024/DI/0422,UT - Max Life Insurance,1502,DC0000YY,Aadhaar Card (Central),1258,RC00W74W,0.0,10.0,30.0,2024-06-19 04:27:07
68579,PID/MAXLI/2024/DI/0422,UT - Max Life Insurance,1502,SH00050P,Pond Construction for SC beneficiaries in plai...,6505,RC00W74W,0.0,10.0,30.0,2024-06-19 04:27:07
68580,PID/MAXLI/2024/DI/0422,UT - Max Life Insurance,1502,SH0006V4,Post Graduate Indira Gandhi Scholarship for Si...,8896,RC00W74W,0.0,10.0,30.0,2024-06-19 04:27:07
68581,PID/MAXLI/2024/DI/0422,UT - Max Life Insurance,1502,SH0000VT,Post Matric Scholarship for Students belonging...,1145,RC00W74W,0.0,10.0,30.0,2024-06-19 04:27:07
68582,PID/MAXLI/2024/DI/0422,UT - Max Life Insurance,1502,SH0000PQ,Post Matric Scholarship for Students with Benc...,926,RC00W74W,0.0,10.0,30.0,2024-06-19 04:27:07


In [137]:
SH0001B3_rate = rate_card[rate_card['schemes_Guid']=='SH0001B3'][['created_on','schemes_Guid','open_price','Docket submitted price','scheme_document_received price']].sort_values('created_on')

In [138]:
SH0001B3_rate.reset_index(drop=True, inplace=True)
SH0001B3_rate

,created_on,schemes_Guid,open_price,Docket submitted price,scheme_document_received price
0,2024-06-19 04:27:07,SH0001B3,0.0,10.0,30.0
1,2024-08-13 18:59:11,SH0001B3,0.0,10.0,30.0
2,2024-08-21 11:11:59,SH0001B3,0.0,0.0,0.0


In [139]:
file_Name = 'cases_report_Max Life Insurance_(All States)_2025-02-28.csv'
data0 = csvORexcel()
data0.head()

,Case Id,Createdon,Scheme/Doc,Scheme/Doc GUID,Status,Docket Submitted Date,Benefit received Date,HD Suspected Cases,Case Organization,Case District,Citizen GUID,Citizen Name,Gender,Mobile,Age,Family GUID,Family Name,HD ID,HD Name,Opsco name,AMS ID
0,EP049YDX,02-Jul-2024,UT-Income Certificate,DC0004SV,Scheme/Document received,01-Oct-2024,01-Oct-2024,Not Suspected,UT - Max Life Insurance,HARIDWAR,PN04AH9H,Johar Singh,M,9.627990e+09,66.0,PG041ANL,Singh,sakshi.katariya,SAKSHI,Renu Sharma,18495.0
1,EP049YNG,02-Jul-2024,UT-Income Certificate,DC0004SV,Scheme/Document received,01-Oct-2024,01-Oct-2024,Not Suspected,UT - Max Life Insurance,HARIDWAR,PN04AHFF,Rajesh Kumar,M,8.532094e+09,52.0,PG041ATO,Kumar,sakshi.katariya,SAKSHI,Renu Sharma,18495.0
2,EP049YPG,02-Jul-2024,UT-Domicile Certificate,DC0004SU,Scheme/Document received,01-Oct-2024,01-Oct-2024,Not Suspected,UT - Max Life Insurance,HARIDWAR,PN04AHHI,Veshnvi Veshnvi,F,9.627048e+09,19.0,PG041AVO,Veshnvi,sakshi.katariya,SAKSHI,Renu Sharma,18495.0
3,EP049YPV,02-Jul-2024,UT-Caste Certificate,DC0004SK,Scheme/Document received,01-Oct-2024,01-Oct-2024,Not Suspected,UT - Max Life Insurance,HARIDWAR,PN04AHI0,Shibi Shibi,F,9.758501e+09,18.0,PG041AW3,Shibi,sakshi.katariya,SAKSHI,Renu Sharma,18495.0
4,EP049YQ6,02-Jul-2024,UT-Domicile Certificate,DC0004SU,Scheme/Document received,01-Oct-2024,01-Oct-2024,Not Suspected,UT - Max Life Insurance,HARIDWAR,PN04AHI0,Shibi Shibi,F,9.758501e+09,18.0,PG041AW3,Shibi,sakshi.katariya,SAKSHI,Renu Sharma,18495.0


In [140]:
# Remove last row.
data0.drop(index = data0[data0.Createdon.isna()].index, inplace=True)

# Replace null values
data0['Scheme/Doc'].fillna('a', inplace=True)
data0['Citizen Name'].fillna('a', inplace=True)
data0['HD Name'].fillna('blank', inplace=True)
data0.Mobile.fillna(0, inplace=True)

# Changing status values and keeping only "Open/Submit/BR"
data0['Status'] = data0['Status'].apply(lambda x: 'Open' if x == 'Data complete' else 'Submitted' if (x=='Docket submitted' or x=='Document ready') else "Benefit Received" if x=='Scheme/Document received' else x)

# Changing Case Organization values from state initials to full state name.
data0['Case Organization'] = data0['Case Organization'].apply(lambda x: states[x[:2]])

# Renaming column "Case Organiisation" & "Case District" to "State" & "Disctrict"
data0.rename(columns={"Case Organization":"State","Case District":"District"}, inplace=True)

# Filling missing values in "Citizen District" column with "Not Captured"
if 'Citizen District' in data0.columns:
    data0['Citizen District'].fillna('Not Captured', inplace=True)

# Removing extra spaces in Opsco name 
data0['Opsco name'] = data0['Opsco name'].apply(lambda x: rem_space(x))

# Convert Mobile column from float to string for concatenation.
data0['Mobile'] = data0['Mobile'].apply(lambda x: str(x).strip())

# Change gender from initial letter to full form.
data0['Gender'] = data0['Gender'].apply(lambda x: 'Male' if x=='M' else 'Female' if x=='F' else 'Other' if x=='O' else x)

# Adding a boolean column to identify if citizen opted for YC
YC_citizens = data0[(data0['Scheme/Doc GUID']=='SH0009RR') & (data0['Status']=='Benefit Received')]['Citizen GUID'].to_list()
data0['isYC'] = data0['Citizen GUID'].apply(lambda x: 'Yes' if x in YC_citizens else 'No')

# Convert "Createdon", "Docket Submitted Date", "Benefit received Date" column data type to Datetime format
dt_col = ['Createdon', 'Createdon.1', 'Docket Submitted Date', 'Benefit received Date', 'DOB']

for col in dt_col:
    try:
        data0[col] = pd.to_datetime(data0[col], format='mixed', errors='ignore')
        #data0[col] = data0[col].apply(lambda x: x.strftime('%d-%m-%Y') if type(x) != pd._libs.tslibs.nattype.NaTType else x)
    except KeyError:
        continue


# Deleting records with status "Case Aborted" and "Application rejected"
rejectedDF = data0[(data0.Status == 'Case Aborted') | (data0.Status == 'Application rejected')] # Storing prev step deleted data
data0 = data0[(data0['Status'] != 'Case Aborted') & (data0['Status'] != 'Application rejected')]

data0.reset_index(inplace=True, drop=True)
data0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15979 entries, 0 to 15978
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Case Id                15979 non-null  object        
 1   Createdon              15979 non-null  datetime64[ns]
 2   Scheme/Doc             15979 non-null  object        
 3   Scheme/Doc GUID        15979 non-null  object        
 4   Status                 15979 non-null  object        
 5   Docket Submitted Date  13686 non-null  datetime64[ns]
 6   Benefit received Date  13072 non-null  datetime64[ns]
 7   HD Suspected Cases     15979 non-null  object        
 8   State                  15979 non-null  object        
 9   District               15979 non-null  object        
 10  Citizen GUID           15979 non-null  object        
 11  Citizen Name           15979 non-null  object        
 12  Gender                 15979 non-null  object        
 13  M

In [141]:
SH0001B3_data = data0[data0['Scheme/Doc GUID'] == 'SH0001B3']
SH0001B3_data.head()

,Case Id,Createdon,Scheme/Doc,Scheme/Doc GUID,Status,Docket Submitted Date,Benefit received Date,HD Suspected Cases,State,District,Citizen GUID,Citizen Name,Gender,Mobile,Age,Family GUID,Family Name,HD ID,HD Name,Opsco name,AMS ID,isYC
483,EP04AESR,2024-07-16,Aadhaar Seeding to Bank Account (Central),SH0001B3,Benefit Received,2024-07-22,2024-07-22,Not Suspected,Uttarakhand,HARIDWAR,PN04ATHU,Neetu Kumar,Male,9639040305.0,39.0,PG041N33,Kumar,Pratibha.Devi,Pratibha Devi,Renu Sharma,9576.0,No
590,EP04AGV9,2024-07-17,Aadhaar Seeding to Bank Account (Central),SH0001B3,Benefit Received,2024-07-22,2024-07-22,Not Suspected,Uttarakhand,HARIDWAR,PN04AUYC,Ahsan Ahsan,Male,8057680730.0,43.0,PG041OLM,Ahsan,mohd.intkhab.ali,Mohd Intkhab Ali,Vikki Kumar,12928.0,No
666,EP04AIY0,2024-07-18,Aadhaar Seeding to Bank Account (Central),SH0001B3,Benefit Received,2024-07-21,2024-07-21,Not Suspected,Uttarakhand,HARIDWAR,PN04AWCF,Tushar Kashyap,Male,7078690996.0,25.0,PG041PXU,Kashyap,Pratibha.Devi,Pratibha Devi,Renu Sharma,9576.0,No
741,EP04AKDC,2024-07-19,Aadhaar Seeding to Bank Account (Central),SH0001B3,Benefit Received,2024-07-21,2024-07-21,Not Suspected,Uttarakhand,HARIDWAR,PN04AX7K,Santosh Santosh,Female,8057395318.0,47.0,PG041QSS,Santosh,Pratibha.Devi,Pratibha Devi,Renu Sharma,9576.0,No
933,EP04AP1N,2024-07-22,Aadhaar Seeding to Bank Account (Central),SH0001B3,Benefit Received,2024-07-31,2024-07-31,Not Suspected,Uttarakhand,HARIDWAR,PN04B02W,Inatasar Inatasar,Male,8057671939.0,43.0,PG041TLN,Inatasar,mohd.intkhab.ali,Mohd Intkhab Ali,Vikki Kumar,12928.0,No


In [142]:
open_price = []
Docket_submitted_price = []
scheme_document_received_price	= []

for idx in data0.index:
    if data0['Scheme/Doc GUID'][idx] not in rate_card['schemes_Guid']:
        open_price.append(0.0)
    else:
        sch_rate_card = rate_card[rate_card['schemes_Guid'] == data['Scheme/Doc GUID'][idx]].reset_index(drop=True)
        # This condition is for 1 rate card update
        if len(sch_rate_card) == 1:
            open_price.append(sch_rate_card['open_price'])
        else:
            # This condition is for 2 or more rate card update
            for i in sch_rate_card.index:
                if data0['Createdon'][idx].date() <= sch_rate_card['created_on'][i].date():
                    open_price.append(sch_rate_card['open_price'][i])
                    break
                elif data0['Createdon'][idx].date() >= sch_rate_card['created_on'].iloc[-1].date():
                    open_price.append(sch_rate_card['open_price'].iloc[-1])
                    break
                elif (data0['Createdon'][idx].date() > sch_rate_card['created_on'][i].date()) & (data0['Createdon'][idx].date() < sch_rate_card['created_on'][i+1].date()):
                    open_price.append(sch_rate_card['open_price'][i])
                    break

for idx in data0.index:
    if data0['Scheme/Doc GUID'][idx] not in rate_card['schemes_Guid']:
        Docket_submitted_price.append(0.0)
    else:
        sch_rate_card = rate_card[rate_card['schemes_Guid'] == data['Scheme/Doc GUID'][idx]].reset_index(drop=True)
        # This condition is for 1 rate card update
        if len(sch_rate_card) == 1:
            Docket_submitted_price.append(sch_rate_card['Docket submitted price'])
        else:
            # This condition is for 2 or more rate card update
            for i in sch_rate_card.index:
                if data0['Docket Submitted Date'][idx].date() <= sch_rate_card['created_on'][i].date():
                    Docket_submitted_price.append(sch_rate_card['Docket submitted price'][i])
                    break
                elif data0['Docket Submitted Date'][idx].date() >= sch_rate_card['created_on'].iloc[-1].date():
                    Docket_submitted_price.append(sch_rate_card['Docket submitted price'].iloc[-1])
                    break
                elif (data0['Docket Submitted Date'][idx].date() > sch_rate_card['created_on'][i].date()) & (data0['Docket Submitted Date'][idx].date() < sch_rate_card['created_on'][i+1].date()):
                    Docket_submitted_price.append(sch_rate_card['Docket submitted price'][i])
                    break

for idx in data0.index:
    if data0['Scheme/Doc GUID'][idx] not in rate_card['schemes_Guid']:
        scheme_document_received_price.append(0.0)
    else:
        sch_rate_card = rate_card[rate_card['schemes_Guid'] == data['Scheme/Doc GUID'][idx]].reset_index(drop=True)
        # This condition is for 1 rate card update
        if len(sch_rate_card) == 1:
            scheme_document_received_price.append(sch_rate_card['scheme_document_received price'])
        else:
            # This condition is for 2 or more rate card update
            for i in sch_rate_card.index:
                if data0['Benefit received Date'][idx].date() <= sch_rate_card['created_on'][i].date():
                    scheme_document_received_price.append(sch_rate_card['scheme_document_received price'][i])
                    break
                elif data0['Benefit received Date'][idx].date() >= sch_rate_card['created_on'].iloc[-1].date():
                    scheme_document_received_price.append(sch_rate_card['scheme_document_received price'].iloc[-1])
                    break
                elif (data0['Benefit received Date'][idx].date() > sch_rate_card['created_on'][i].date()) & (data0['Benefit received Date'][idx].date() < sch_rate_card['created_on'][i+1].date()):
                    scheme_document_received_price.append(sch_rate_card['scheme_document_received price'][i])
                    break

data0['open_price'] = open_price
data0['Docket submitted price'] = Docket_submitted_price
data0['scheme_document_received price'] = scheme_document_received_price
data0['HD_Payment'] = data0.open_price + data0['Docket submitted price'] + data0['scheme_document_received price'] # Adding column for HD payment.

In [1]:
Successful = []
Unsuccessful = []
already= []
Mobile_Issue = []
other = []
logs = open(r"C:\Users\akash\Downloads\1-Open Data New - 3-7-2025.txt",'r', encoding='utf-8')
for log in logs:
    if log == "\n":
        continue
    elif " successful." in log:
        Successful.append(log)
    elif " already " in log:
        already.append(log)
    elif " unsuccessful." in log:
        Unsuccessful.append(log)
    elif "mobile number" in log:
        Mobile_Issue.append(log)
    else:
        other.append(log)
logs.close()

In [2]:
len(Successful)

231

In [3]:
len(Unsuccessful)

2

In [4]:
Unsuccessful[:10]

['34 Sanjay Hemu Rathod 9011802268 Data Upload unsuccessful.\n',
 '68 Vishal Manohar Rathod 9370417144 Data Upload unsuccessful.\n']

In [5]:
len(already)

0

In [6]:
len(Mobile_Issue)

0

In [7]:
len(other)

0

In [8]:
len(Successful)+len(Unsuccessful)+len(Mobile_Issue)+len(already)

233